In [8]:
import pandas as pd
import numpy as np

from autogluon.tabular import TabularDataset, TabularPredictor
from dotenv import dotenv_values

In [2]:
config = dotenv_values('../.env')
train = pd.read_parquet(config["ENGINEERED_DATA"] + "train_fe.parquet")
train.drop('customer_ID', axis=1, inplace=True)
train.head()

,P_2_mean,P_2_std,P_2_min,P_2_max,P_2_last,D_39_mean,D_39_std,D_39_min,D_39_max,D_39_last,...,D_64_count,D_64_last,D_64_nunique,D_66_count,D_66_last,D_66_nunique,D_68_count,D_68_last,D_68_nunique,target
0,0.933824,0.024194,0.868580,0.960384,0.934745,0.230769,0.832050,0,3,0,...,13,0,1,13,-1,1,13,6,1,0
1,0.899820,0.022119,0.861109,0.929122,0.880519,7.153846,6.743468,0,19,6,...,13,0,1,13,-1,1,13,6,1,0
2,0.878454,0.028911,0.797670,0.904482,0.880875,0.000000,0.000000,0,0,0,...,13,2,1,13,-1,1,13,6,1,0
3,0.598969,0.020107,0.567442,0.623392,0.621776,1.538462,3.017046,0,9,0,...,13,0,1,13,-1,1,13,3,3,0
4,0.891679,0.042325,0.805045,0.940382,0.871900,0.000000,0.000000,0,0,0,...,13,0,1,13,1,1,13,6,1,0


In [3]:
save_path = 'agModels-predictClass'  # specifies folder to store trained models
predictor = TabularPredictor(label='target', 
                             problem_type='regression',
                             path=save_path).fit(train, 
                                                 presets='optimize_for_deployment',
                                                 hyperparameters= {'NN_TORCH':{}, 'FASTAI':{},
                                                                   'GBM': [{'extra_trees': True,
                                                                            'ag_args': {'name_suffix': 'XT'}}, {}, 'GBMLarge',], 
                                                                   'CAT': {}, 
                                                                   'XGB': {},
                                                                  },
                                                 holdout_frac=0.5, 
                                                 time_limit=3600)

Presets specified: ['optimize_for_deployment']
Beginning AutoGluon training ... Time limit = 3600s
AutoGluon will save models to "agModels-predictClass\"
AutoGluon Version:  0.5.2
Python Version:     3.9.12
Operating System:   Windows
Train Data Rows:    458913
Train Data Columns: 918
Label Column: target
Preprocessing data ...
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    4095.7 MB
	Train Data (Original)  Memory Usage: 1854.93 MB (45.3% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
			Note: Converting 67 features to boolean dtype as they only contain 2 unique values.
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenera

[1000]	valid_set's rmse: 0.265076


	-0.265	 = Validation score   (-root_mean_squared_error)
	194.29s	 = Training   runtime
	4.68s	 = Validation runtime
Fitting model: LightGBM ... Training model for up to 3366.26s of the 3366.25s of remaining time.
	Not enough memory to train LightGBM... Skipping this model.
Fitting model: CatBoost ... Training model for up to 3362.96s of the 3362.95s of remaining time.
C:\Users\Kaleb\anaconda3\envs\myenv\lib\site-packages\xgboost\compat.py:31: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
	-0.2645	 = Validation score   (-root_mean_squared_error)
	708.47s	 = Training   runtime
	0.49s	 = Validation runtime
Fitting model: NeuralNetFastAI ... Training model for up to 2653.95s of the 2653.94s of remaining time.
	Not enough memory to train NeuralNetFastAI... Skipping this model.
Fitting model: XGBoost ... Training model for up to 2650.69s of t

[1000]	valid_set's rmse: 0.265228


	-0.2652	 = Validation score   (-root_mean_squared_error)
	220.73s	 = Training   runtime
	3.96s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ... Training model for up to 360.0s of the 1845.17s of remaining time.
	-0.2633	 = Validation score   (-root_mean_squared_error)
	1.99s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 1760.77s ... Best model: "WeightedEnsemble_L2"
Deleting model XGBoost. All files under agModels-predictClass\models\XGBoost\ will be removed.
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("agModels-predictClass\")


In [9]:
def amex_metric_mod(y_true, y_pred):
    labels     = np.transpose(np.array([y_true, y_pred]))
    labels     = labels[labels[:, 1].argsort()[::-1]]
    weights    = np.where(labels[:,0]==0, 20, 1)
    cut_vals   = labels[np.cumsum(weights) <= int(0.04 * np.sum(weights))]
    top_four   = np.sum(cut_vals[:,0]) / np.sum(labels[:,0])

    gini = [0,0]
    for i in [1,0]:
        labels         = np.transpose(np.array([y_true, y_pred]))
        labels         = labels[labels[:, i].argsort()[::-1]]
        weight         = np.where(labels[:,0]==0, 20, 1)
        weight_random  = np.cumsum(weight / np.sum(weight))
        total_pos      = np.sum(labels[:, 0] *  weight)
        cum_pos_found  = np.cumsum(labels[:, 0] * weight)
        lorentz        = cum_pos_found / total_pos
        gini[i]        = np.sum((lorentz - weight_random) * weight)

    return 0.5 * (gini[1]/gini[0] + top_four)

In [5]:
predictor = TabularPredictor.load("agModels-predictClass/")

In [6]:
preds = predictor.predict(train[train.columns[:-1]])

In [10]:
print(amex_metric_mod(train["target"].values, preds))

0.8468989226094761


In [11]:
del train

In [13]:
test = pd.read_parquet(config["ENGINEERED_DATA"] + "test_fe.parquet")
test[test.columns[1:]].head()

,P_2_mean,P_2_std,P_2_min,P_2_max,P_2_last,D_39_mean,D_39_std,D_39_min,D_39_max,D_39_last,...,D_63_nunique,D_64_count,D_64_last,D_64_nunique,D_66_count,D_66_last,D_66_nunique,D_68_count,D_68_last,D_68_nunique
0,0.601387,0.020190,0.568930,0.631315,0.568930,2.222222,3.527668,0,8,4,...,1,9,3,2,9,-1,1,9,6,2
1,0.862166,0.031436,0.794469,0.913501,0.841177,5.076923,6.034091,0,17,4,...,1,13,0,1,13,-1,1,13,6,1
2,0.748955,0.061456,0.673112,0.835114,0.697522,6.000000,9.000000,0,23,0,...,1,13,3,2,13,1,1,13,4,2
3,0.474728,0.028856,0.428457,0.514222,0.513186,15.846154,4.355957,7,23,11,...,1,13,2,1,13,-1,1,13,5,1
4,0.324100,0.049865,0.254478,0.425764,0.254478,11.846154,6.681394,1,26,26,...,1,13,2,2,13,-1,1,13,5,2


In [14]:
preds = predictor.predict(test[test.columns[1:]])

In [15]:
submit = pd.read_csv(config["SAMPLE_PATH"])
submit['prediction'] = preds
submit.to_csv(config["SUBMISSION_FOLDER"] + f'auto_gluon.csv', index=None)